In [27]:
import json
from pathlib import Path

import albumentations as A
import numpy as np
import pandas as pd
from PIL import Image

In [2]:
# width or x - горизонталь
# height or y - вертикаль
# Each row is - class x_center y_center width height format.
# Box coordinates must be in normalized xywh format (from 0 - 1). If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
# Class numbers are zero-indexed (start from 0).
# пример строки в файле .txt - '45 0.479492 0.688771 0.955609 0.5955\n'

In [3]:
labels_dir = Path('datasets/coco128/labels/train2017')

def get_files(files_dir: Path, suff: str = 'txt') -> list[Path]:
    return sorted(Path(files_dir).glob(f'*.{suff}'))

labels = get_files(labels_dir)

with open(labels[0], 'r') as label:
    data = label.readlines()

data

['45 0.479492 0.688771 0.955609 0.5955\n',
 '45 0.736516 0.247188 0.498875 0.476417\n',
 '50 0.637063 0.732938 0.494125 0.510583\n',
 '45 0.339438 0.418896 0.678875 0.7815\n',
 '49 0.646836 0.132552 0.118047 0.0969375\n',
 '49 0.773148 0.129802 0.0907344 0.0972292\n',
 '49 0.668297 0.226906 0.131281 0.146896\n',
 '49 0.642859 0.0792187 0.148063 0.148062\n']

# Преобразование координат объектов в формат yolo

## Загрузка координат в pandas

In [5]:
location_csv = Path('../train_dataset_train/train.csv')

In [6]:
location = pd.read_csv(location_csv)

In [12]:
location

,ID_img,count_region,region_shape
0,3376.JPG,0.0,0.0
1,3377.JPG,0.0,0.0
2,3378.JPG,0.0,0.0
3,3379.JPG,0.0,0.0
4,3380.JPG,0.0,0.0
...,...,...,...
5059,8435.JPG,0.0,0.0
5060,8436.JPG,0.0,0.0
5061,8437.JPG,0.0,0.0
5062,8438.JPG,0.0,0.0


In [14]:
location[location['region_shape'] != '0.0']

,ID_img,count_region,region_shape
395,3771.JPG,1.0,"['{""cx"":2715,""cy"":419,""r"":133}']"
409,3785.JPG,1.0,"['{""cx"":2719,""cy"":2097,""r"":75}']"
429,3805.JPG,1.0,"['{""cx"":3220,""cy"":1280,""r"":185}']"
523,3899.jpg,1.0,"['{""cx"":4125,""cy"":2129,""r"":80}']"
675,4051.JPG,5.0,"['{""cx"":2396,""cy"":1187,""r"":111}', '{""cx"":2765,..."
703,4079.jpg,4.0,"['{""cx"":3193,""cy"":1598,""r"":71}', '{""cx"":2102,""..."
722,4098.JPG,2.0,"['{""cx"":3404,""cy"":3006,""r"":80}', '{""cx"":3284,""..."
787,4163.JPG,2.0,"['{""cx"":3321,""cy"":924,""r"":50}', '{""cx"":3277,""c..."
794,4170.jpg,1.0,"['{""cx"":3234,""cy"":3261,""r"":80}']"
806,4182.JPG,5.0,"['{""cx"":3197,""cy"":2647,""r"":60}', '{""cx"":3237,""..."


In [63]:
def get_size(image_file: Path) -> tuple[int, int]:
    forest_image = Image.open(image_file)
    width, height = forest_image.size
    return width, height

In [15]:
def get_fraction(image_size: tuple[int, int], location: list[dict[str, int]]) -> list[list[float]]:
    # location consist: 
    # ‘r’ - область внутри которой находиться человек
    # ‘cx’ - центр окружности по координате x
    # ‘cy’ - центр окружности по координате y
    boxes = []
    width, height = image_size
    for box in location:
        center_x = str(box['cx'] / width)
        center_y = str(box['cy'] / height)
        box_width = str(box['r'] / width)
        box_height = str(box['r'] / height)
        boxes.append([0, center_x, center_y, box_width, box_height])
    return boxes


In [69]:
def get_text(boxes: list[list[float]]) -> list[str]:
    boxes_str = []
    for location in boxes:
        boxes_str.append(' '.join(location))
    return boxes_str

In [64]:
image_file = Path('../train_dataset_train/train/3771.JPG')
size = get_size(image_file)
size

(5472, 3078)

In [71]:
coord = location.iloc[395]['region_shape']
boxes = json.loads(coord.replace("\'", ""))
boxes = get_fraction(size, boxes)
boxes = get_text(boxes)
boxes

TypeError: sequence item 0: expected str instance, float found

In [ ]:
' '.join()

# Аугументация датасета